# 7.e Sieci Konwolucyjne (CNN)

Naszym zadaniem będzie analiza wydźwięku (sentiment analysis), na zbiorze danych pochodzącym z konkursu PolEval 2019. Pracujemy na korpusie tweetów, a więc tekstów o ograniczonej długości, i staramy się zbudować narzędzie do wykrywania cyberbullyingu.

Skonstruujemy sieć opartą o warstwę konwolucyjną, i klasyfikator binarny. Ponieważ pracujemy z konwolucjami, zalecane jest wykorzystanie GPU.

In [ ]:
!wget https://github.com/sagespl/nlp-masterclass/blob/main/modu%C5%82-07/cnn_data.zip?raw=true
!mv cnn_data.zip?raw=true cnn_data.zip
!unzip cnn_data.zip

#### POBIERANIE MODELU FASTTEXT

Może potrwać do 10 minut

Korzystamy z embeddingów fasttext, aby uodpornić się na złą pisowanię, lub internetowe słowotwórstwo

In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.pl.300.bin.gz
!gunzip cc.pl.300.bin.gz
!python3 -m pip install fasttext

#### Pobranie pakietów

In [ ]:
import os
import random
import numpy
import torch
from tqdm.notebook import tqdm

#### Wczytanie danych

In [ ]:
def load_data(path):
    with open(path) as f:
        data_text = f.read()
    docs = data_text.split("\n")
    out = []
    for datapoint in docs:
        text, label = datapoint.split("\t")
        toks = text.split() # prosta tokenizacja po białych znakach. 
        # Moglibyśmy zastosować tokenizację nltk.tokenize.TweetTokenizer
        # ale przedłuża to uczenie, i nie poprawia znacząco wyników
        out.append((toks, int(label)))
    return out


train_data = load_data(os.path.join("cnn_data", "train.tab"))
dev_data = load_data(os.path.join("cnn_data", "dev.tab"))
test_data = load_data(os.path.join("cnn_data", "test.tab"))
label_list = [0,1]
num_labels = len(label_list)

print("Kategorie dla klasyfikatora: ", label_list, "\n")
print("Długość zbioru treningowego (liczona w dokumentach): ", len(train_data))
print("Przykładowy dokument:")
print(train_data[100])
doc_lens = [len(dp[0]) for dp in train_data+dev_data+test_data]
print("Maksymalna długość dokumentu: ", max(doc_lens))
print("Średnia długość dokumentu: ", sum(doc_lens)/len(doc_lens))

max_len = max(doc_lens)

#### Przygotowanie zanurzeń słów

(Może potrwać kilka minut)

In [ ]:
import fasttext
VEX = fasttext.load_model("cc.pl.300.bin")
num_feats = VEX.get_dimension()

def w2v(form):
    try:
        return VEX.get_word_vector(form)
    except KeyError:
        return numpy.zeros((num_feats,))

Praca z embeddingami fasttext: embeddingi dla słów spoza słownika

In [ ]:
print(VEX.get_nearest_neighbors('jesieniara'))
print(VEX.get_subwords("jesieniara"))

#### Ustawienie wsparcia dla karty graficznej

In [ ]:
if torch.cuda.is_available():  
    device = "cuda:0" 
else:  
    device = "cpu"  
DEVICE = torch.device(device)  

#### Konstrukcja sieci neuronowej

In [ ]:
from torch import nn, cat, tanh

class ConvolutionalModel(nn.Module):
    def __init__(self, num_feats, filters_num, window_size, pool_ratio, seq_len, output_size):
        super(ConvolutionalModel, self).__init__()
        self.padding_width = int((window_size -1)/2) # długość paddingu (z każdej ze stron), dla skrajnych elementów
        self.seq_len = seq_len
        self.conv = nn.Conv1d(num_feats, filters_num, window_size, padding=self.padding_width) # warstwa konwolucyjna
        # argumenty to liczba kanałów, liczba filtrów, szerokośc okna, szerokośc paddingu
        self.relu = nn.ReLU() # funkcja aktywacji 
        self.pool = nn.MaxPool1d(pool_ratio) # podpróbkowanie po warstwie konwolucyjnej
        self.convout_dim = filters_num * (seq_len // pool_ratio) # długość wyjścia po konwolucji i poolingu
        self.dense = nn.Linear(self.convout_dim, output_size) # klasyfikator
        self.softmax = nn.LogSoftmax(dim=1) # funkcja aktywacji klasyfikatora

    def forward(self, input):
        convolved = self.relu(self.conv(input)) # przejście przez konwolucję i funkcję aktywacji
        pooled = self.pool(convolved) # redukcja wymiarowości przez podpróbkowanie
        flattened = pooled.reshape(1, -1) # spłaszczenie (konkatenacja outputu z różnych filtrów)
        output = self.softmax(self.dense(flattened)) # klasyfikator
        return output

#### Definicja funkcji pomocniczych

Definiujemy funkcję zamieniającą dane w przykłady dla sieci, trenującą, i testującą na tych przykładach oraz interpretującą wyjście z sieci.

Stosujemy podwójny padding, tj. :
    - padding wszystkich danych do jednej długości (najdłuższego w tokenach tweeta)
    - padding tak przetworzonych danych, dla reprezentatywności danych z krawędzi sekwencji

In [ ]:
from torch import tensor

padding_vector = numpy.zeros((num_feats,))

def datapoint_to_training_example(datapoint, max_len):
    tokens, label = datapoint
    forms_vec = []
    for form in tokens:
        forms_vec.append(w2v(form)) # reprezentacja tokenów jako wektorów
    while len(forms_vec) < max_len:
        forms_vec.insert(0, padding_vector) # padding aby wyrównać przykłady co do długości
    X = torch.tensor(forms_vec, dtype=torch.float32)
    X = X.transpose(0,1) # transpozycja, aby numery cech były pierwszą współrzędną, a numery tokenów drugą
    X = X.unsqueeze(0).to(DEVICE) # przekształcanie tensorów, dodanie sztucznego wymiaru dla batchów
    Y = torch.tensor([label]).to(DEVICE)
    return X, Y


def train_on_example(model, criterion, X, Y):
    model.zero_grad() # wyzerowanie gradientów
    output = model(X) # przejście przez sieć
    loss = criterion(output, Y) # obliczenie straty
    loss.backward() # wsteczna propagacja
    for p in model.parameters(): # uaktualnienie parametrów
        p.data.add_(p.grad.data, alpha = -learning_rate)
    return loss.item()


def out_to_label(out):
    index = out.topk(1).indices # wybieramy indeks o najwyższej wartości
    tag = label_list[index] # odczytujemy interpretację indeksu
    return tag, index
    
    
def test_on_example(model, X, Y):
    output = model(X)
    decision = output.topk(1).indices[0]
    tp, fp, tn, fn = 0, 0, 0, 0
    if decision == 1 and Y == 1:
        tp = 1
    elif decision == 1 and Y == 0:
        fp = 1
    elif decision == 0 and Y == 0:
        tn = 1
    elif decision == 0 and Y == 1:
        fn = 1
    return tp, fp, tn, fn


#### Przykładowe przejście przez sieć

In [ ]:
from torch import nn, cat, tanh

X, Y = datapoint_to_training_example(train_data[0], max_len)
print("Kształt tensora reprezentującego wejście: ", X.shape) # batch, cechy z embeddingów, liczba tokenów
seq_len = X.shape[2]

filters_num = 40 # liczba filtrów
window_size = 3 # szerokość okna, w tokenach
padding_width = int((window_size -1)/2) # obliczamy długość paddingu korzystając z wzoru
conv = nn.Conv1d(num_feats, filters_num, window_size, padding = padding_width).to(DEVICE)
convout = conv(X)
print("Kształt wyjścia z warstwy konwolucjnej:", convout.shape)
relu = nn.ReLU()
activated = relu(convout) # funkcja aktywacji po konwolucji


pool_ratio = 4
pool = nn.MaxPool1d(pool_ratio)
pooled = pool(activated)
print("Kształt po poolingu: ", pooled.shape)
flattened = pooled.reshape(1, -1) # spłaszczanie poprzez łączenie outputu z różnych filtrów
print("Kształt po spłaszczeniu: ", flattened.shape)

convout_dim = filters_num * (seq_len // pool_ratio) # obliczenie długości wejścia do klasyfikatora
output_size = 2
classifier = nn.Linear(convout_dim, output_size).to(DEVICE)
softmax = nn.LogSoftmax(dim=1)
classified = softmax(classifier(flattened))
print("Kształt wyjścia z sieci: ", classified.shape)

#### Augmentacja danych
Dane są bardzo nierówne, przykłady pozytywne (tweety zawierające cyberbullying) to tylko 8% całości danych. To bardzo poważny problem, wymagający modyfikacji w procesie uczenia, i w procesie ewaluacji. Sieć zawsze stwierdająca brak cyberbullyingu, uzyskałaby 92% poprawności.

W tym celu stosujemy prostą sztuczkę do augmentacji danych, każdy pozytywny przykład kopiujemy w danych 10 razy, aby wyrównać klasy, poprawia to wyniki modelu.

Ze względu na zadanie postawione przed siecią, lepiej będzie używać miar innych niż accuracy:

    precision = TP/TP+FP
    
    recall = TP/TP+FN
    
Gdzie pozytywny wynik to tweet zawierający cyberbullying.

In [ ]:
pos_examples = [ex for ex in train_data if ex[1] == 1]
pos_proportion = 100 * len(pos_examples)/len(train_data) 
print("proporcja przykładów pozytywnych: {:4.2f}%".format(pos_proportion))

augmented_train_data = train_data + 10 * pos_examples
random.shuffle(augmented_train_data)
pos_proportion = 100 * (11 * len(pos_examples))/len(augmented_train_data) 
print("proporcja przykładów pozytywnych po augmentacji: {:4.2f}%".format(pos_proportion))

### Trening i ewaluacja danych

Dane trenujemy przez dziesięć epok, po każdej epoce ewaluujemy model na devsecie. Po całym procesie, ewaluujemy model na danych testowych.

Model uzyskuje wyniki porównywalne z innymi wynikami w konkursie

In [ ]:
model = ConvolutionalModel(num_feats, 100, 5, 5, max_len, num_labels).to(DEVICE) # 100 filtrów, 5 tokenów szerokości okna, współczynnik poolingu 5
criterion = torch.nn.NLLLoss()
learning_rate = 0.0005
epochs = 10

for epoch in range(epochs):
    print("Epoka: ", epoch+1)
    total_loss = 0
    for doc in tqdm(augmented_train_data):
        X, Y = datapoint_to_training_example(doc, max_len)
        loss = train_on_example(model, criterion, X, Y)
        total_loss += loss
    print(total_loss)
        

    tp, fp, tn, fn = 0, 0, 0, 0
    with torch.no_grad(): # wyłączamy akumulację gradientów dla ewaluacji
        for doc in tqdm(dev_data):
            X, Y = datapoint_to_training_example(doc, max_len)
            tpd, fpd, tnd, fnd = test_on_example(model, X, Y) # obliczamy miary ewaluacyjne
            tp += tpd
            fp += fpd
            tn += tnd
            fn += fnd
        try:
            precision = tp / (fp + tp)
        except ZeroDivisionError:
            precision = 0
        try:
            recall = tp / (fn + tp)
        except ZeroDivisionError:
            recall = 0
        try:
            f1 = 2 * (precision*recall)/(precision+recall)
        except ZeroDivisionError:
            f1 = 0
        print("precision: {:4.2f}%".format(precision * 100))
        print("recall: {:4.2f}%".format(recall * 100))
        print("f1: {:4.2f}%".format(f1 * 100))
        
with torch.no_grad():
    for doc in tqdm(test_data):
        X, Y = datapoint_to_training_example(doc, max_len)
        tpd, fpd, tnd, fnd = test_on_example(model, X, Y)
        tp += tpd
        fp += fpd
        tn += tnd
        fn += fnd
    try:
        precision = tp / (fp + tp)
    except ZeroDivisionError:
        precision = 0
    try:
        recall = tp / (fn + tp)
    except ZeroDivisionError:
        recall = 0
    try:
        f1 = 2 * (precision*recall)/(precision+recall)
    except ZeroDivisionError:
        f1 = 0
    print("\nWyniki na danych testowych:\n")
    print("precision: {:4.2f}%".format(precision * 100))
    print("recall: {:4.2f}%".format(recall * 100))
    print("f1: {:4.2f}%".format(f1 * 100))
